In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#Welcome to my notebook about Titanic contest. 
#First of all, let's save the datasets into variables in order to use it later.
train_titanic = pd.read_csv('/kaggle/input/titanic/train.csv')
test_titanic = pd.read_csv('/kaggle/input/titanic/test.csv')
submission_template = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
#The first step within any project of Machine Learning, is read, analyse and understand the datasets.
#I can see that the columns called 'Age' and 'Cabin', have missing values (nan or empty), so we need to clean data or take decisions about the features for our model
train_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
#We can see the column 'Age' and 'Cabin' have missing values, as 'Cabin' could not be within our features, we can dismiss it.
#The 'Age' column is important for us, so we need to see how we can get rid of the missing values.
train_titanic['Age'].isnull().sum().sum()

177

In [5]:
#In the column 'Age' we have 177 rows with missing values, we can erase those files but that represent 20% of the data so we could lose precision.
#As our first option, we can fill those missing values with the mean of that column.
train_titanic['Age'] = train_titanic['Age'].replace(np.nan, train_titanic['Age'].mean())
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
#Now we've changed the missing values for the mean in 'Age' columns, let's split our data.
#The prediction target is: 'Survived'
y = train_titanic.Survived

#We need to choose the feature data in order to train the model.
feature_data = ['Pclass','Sex','Age','SibSp','Parch']
X = train_titanic[feature_data]

In [7]:
#The values on 'Age' are float, let's convert it to int.
X= X.round(0).astype({'Age':'int'})

In [8]:
#We see that the feature called 'Sex' is a categorical variable, we're going to change it into numerical variable male = 0 and female = 1.
#Having only numerical categories will improve the execution time of the ML models.
X['Sex'] = X['Sex'].replace(['male','female'],[0,1])

In [9]:
#In order to validate the model and get the error, we split our data into training and validation.
#from sklearn.model_selection import train_test_split
#train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

In [10]:
#We're going to use Random forest as our ML model to predict the data.
from sklearn.ensemble import RandomForestRegressor
titanic_model = RandomForestRegressor(random_state=1)
#Let's fit the model
titanic_model.fit(X, y)

RandomForestRegressor(random_state=1)

In [11]:
#Now we take the test dataset
test_titanic['Age'] = test_titanic['Age'].replace(np.nan, test_titanic['Age'].mean())
test_titanic= test_titanic.round(0).astype({'Age':'int'})
test_titanic['Sex'] = test_titanic['Sex'].replace(['male','female'],[0,1])
test_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          418 non-null    int64  
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(1), int64(6), object(4)
memory usage: 36.0+ KB


In [12]:
test_X = test_titanic[feature_data]
test_X

,Pclass,Sex,Age,SibSp,Parch
0,3,0,34,0,0
1,3,1,47,1,0
2,2,0,62,0,0
3,3,0,27,0,0
4,3,1,22,1,1
...,...,...,...,...,...
413,3,0,30,0,0
414,1,1,39,0,0
415,3,0,38,0,0
416,3,0,30,0,0


In [13]:
#Let's predict how many people survived
survived_array = titanic_model.predict(test_X).round(0).astype(int)

In [14]:
survived_array

array([0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [15]:
#Let's calculate the Mean Absolute Error.
#from sklearn.metrics import mean_absolute_error
#survived_error = mean_absolute_error(survived_array, )
#print('The mean absolute error is: {}'.format(survived_error))

In [16]:
submission_template['Survived'] = survived_array

In [17]:
submission_template

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
